In [ ]:
import gensim 
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional, TimeDistributed,GlobalMaxPool1D,Masking
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import json
from sklearn.model_selection import KFold
from keras.regularizers import l1,l2
from keras import regularizers
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/max-labels-intense-selection.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
docs = [x[0] + ' '.join([j[1] for j in nltk.pos_tag(x[0].split())]) for x in data]

In [ ]:
f = open("doc3.txt", "w")
f.write(str(docs))
f.close()

In [ ]:
docs[0]

'I caught Quinn roll his eyes.PRP VBD NNP VB PRP$ NN'

In [ ]:
X = ([x[0] for x in data])

y = np.array([x[1] for x in data])

NameError: ignored

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(y)

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
sentence_lines = list()
for line in X:
  token = word_tokenize(line)
  token = [w.lower() for w in token]
  table = str.maketrans('','',string.punctuation)
  striped = [w.translate(table) for w in token]
  words = [word for word in striped if word.isalpha()]
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  sentence_lines.append(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(sentence_lines)

20000

In [ ]:
model1 = gensim.models.Word2Vec(sentences=sentence_lines, size=100, min_count=1, window=5, iter=100)

In [ ]:
len_words = list(model1.wv.vocab)
print(len(len_words))

27656


In [ ]:
model1.save('model1.model')
model1.save("model1.bin")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
file = 'train_model.txt'
model1.wv.save_word2vec_format(file,binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model1.wv.most_similar('good')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('healer', 0.42116662859916687),
 ('roofed', 0.4152277112007141),
 ('cutter', 0.4065130054950714),
 ('excellent', 0.40426725149154663),
 ('gymnastics', 0.4030221104621887),
 ('edifices', 0.3968561887741089),
 ('breakers', 0.39475059509277344),
 ('worth', 0.3926602602005005),
 ('gulfport', 0.3923709988594055),
 ('commodious', 0.3915606737136841)]

In [ ]:
file = 'train_model.txt'
f=open(file,encoding='utf-8')

In [ ]:
embedding_index={}
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embedding_index[word] = coefs

In [ ]:
tokenize = Tokenizer()
tokenize.fit_on_texts(sentence_lines)
sequences = tokenize.texts_to_sequences(sentence_lines)

In [ ]:
word_index = tokenize.word_index

In [ ]:
sentence_pad = sequence.pad_sequences(sequences,maxlen=200)

In [ ]:
print(sentence_pad.shape)
print(y.shape)

(20000, 200)
(20000,)


In [ ]:
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words,100))
for word,i in word_index.items():
  if i >num_words:
    continue
  embedding_vector = embedding_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
from keras.initializers import Constant
model = Sequential()
embedding_layer = Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),input_length=200,trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(LSTM(256,kernel_regularizer=l2(0.01),recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, dropout=0.25, recurrent_dropout=0)))
model.add(Bidirectional(LSTM(256,kernel_regularizer=l2(0.01),recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, dropout=0.25, recurrent_dropout=0)))
model.add(GlobalMaxPool1D())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          2765700   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 512)          731136    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 512)          1574912   
_________________________________________________________________
global_max_pooling1d (Global (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [ ]:
model.fit(sentence_pad,y,validation_split=0.2,epochs=10,batch_size=128)

Epoch 1/10
125/125 [==============================] - 22s 179ms/step - loss: -78.8994 - accuracy: 0.0851 - val_loss: -171.7208 - val_accuracy: 0.0740
Epoch 2/10
125/125 [==============================] - 22s 173ms/step - loss: -309.3494 - accuracy: 0.0757 - val_loss: -519.8169 - val_accuracy: 0.0000e+00
Epoch 3/10
125/125 [==============================] - 22s 174ms/step - loss: -1128.4723 - accuracy: 0.1177 - val_loss: -1896.4828 - val_accuracy: 0.1293
Epoch 4/10
125/125 [==============================] - 22s 176ms/step - loss: -2847.2161 - accuracy: 0.1850 - val_loss: -3516.6003 - val_accuracy: 0.3203
Epoch 5/10
125/125 [==============================] - 22s 176ms/step - loss: -4088.0596 - accuracy: 0.1891 - val_loss: -5370.4951 - val_accuracy: 0.2072
Epoch 6/10
125/125 [==============================] - 22s 177ms/step - loss: -6340.8115 - accuracy: 0.1700 - val_loss: -7834.7095 - val_accuracy: 0.3370
Epoch 7/10
125/125 [==============================] - 22s 178ms/step - loss: -10013